 - try bigram
 - [x] use pretraining embeddings

In [401]:
import pandas as pd
import os
import numpy as np

In [402]:
# with open(os.path.join(input_path, 'dev_text.txt'), 'r', encoding='utf-8') as f:
#     dev_text = f.read().strip().split('\n')

# with open(os.path.join(input_path, 'heldout_text.txt'), 'r', encoding='utf-8') as f:
#     heldout_text = f.read().strip().split('\n')

In [403]:
# dev_label_path = os.path.join(input_path,'dev_label.txt')
# with open(dev_label_path, 'r', encoding='utf-8') as f:
#     dev_y = f.read().split('\n')

# Prepare data

In [404]:
# dev_data = pd.DataFrame({'text':dev_text, 'label':dev_y})

# dev_data.to_csv(os.path.join(input_path, 'dev_data.tsv'), sep='\t',index=False)

In [405]:
# test_data = pd.DataFrame({'text':heldout_text})
# test_data.to_csv(os.path.join(input_path, 'test_data.tsv'), sep='\t',index=False)

In [633]:
import paths
import baseline_lstm_model
import oh_lstm_model
import fasttext_model
import config
import util
import importlib

In [634]:
reload_packages = [paths, baseline_lstm_model, fasttext_model, oh_lstm_model, util, config]
for package in reload_packages:
    importlib.reload(package)
# importlib.reload(data)

In [635]:
config.batch_size

2

# Load data

In [636]:
import torch
from torchtext import data
import random
from torchtext.data import TabularDataset
import torch.nn as nn
import torch.optim as optim
import spacy
import re



SEED = 1234

torch.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True

nlp = spacy.load('en',disable=['parser', 'tagger', 'ner'])
def tokenizer(string): 
    return [word.text.lower() for word in nlp(clean(string))]

def clean(text):
    '''remove non alphanumeric character, remove links'''
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text)
    text = re.sub(r'https?:/\/\S+', ' ', text)
    return text.strip()

TEXT = data.Field(sequential=True, tokenize = tokenizer, include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [637]:
dev_datafields = [("text", TEXT), ("label", LABEL)]
dev_dataset = TabularDataset(
               path=os.path.join(paths.input_path, 'dev_data.tsv'),
               format='tsv',
               skip_header=True,
               fields=dev_datafields)

test_datafields = [("text", TEXT)]
test_dataset = TabularDataset(
           path=os.path.join(paths.input_path, 'test_data.tsv'),
           format='csv',
           skip_header=True,
           fields=test_datafields)

In [638]:
train_dataset, valid_dataset = dev_dataset.split(split_ratio=0.8, random_state = random.seed(SEED))

In [639]:
TEXT.build_vocab(train_dataset, 
                 max_size = 40000, 
                 vectors = "glove.840B.300d",
                 unk_init = torch.Tensor.normal_ # initialize unk and pad with normal distribution
                )
LABEL.build_vocab(train_dataset)

In [640]:
TEXT.vocab.itos[1]

'<pad>'

In [641]:
TEXT.vocab.vectors[TEXT.vocab.stoi['<unk>']]

tensor([-0.1117, -0.4966,  0.1631, -0.8817,  0.0539,  0.6684, -0.0597, -0.4675,
        -0.2153,  0.8840, -0.7584, -0.3689, -0.3424, -1.4020,  0.3206, -1.0219,
         0.7988, -0.0923, -0.7049, -1.6024,  0.2891,  0.4899, -0.3853, -0.7120,
        -0.1706, -1.4594,  0.2207,  0.2463, -1.3248,  0.6970, -0.6631,  1.2158,
        -1.4949,  0.8810, -1.1786, -0.9340, -0.5675, -0.2772, -2.1834,  0.3668,
         0.9380,  0.0078, -0.3139, -1.1567,  1.8409, -1.0174,  1.2192,  0.1601,
         1.5985, -0.0469, -1.5270, -2.0143, -1.5173,  0.3877, -1.1849,  0.6897,
         1.3232,  1.8169,  0.6808,  0.7244,  0.0323, -1.6593, -1.8773,  0.7372,
         0.9257,  0.9247,  0.1825, -0.0737,  0.3147, -1.0369,  0.2100,  0.6144,
         0.0628, -0.3297, -1.7970,  0.8728,  0.7670, -0.1138, -0.9428,  0.7540,
         0.1407, -0.6937, -0.6159, -0.7295,  0.4308,  0.2862, -0.2481,  0.2040,
         0.8519, -1.4102, -0.1071, -0.8018,  0.2771,  2.5599, -1.6952,  0.1885,
         0.7388,  1.5903, -0.1947, -0.24

In [642]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_dataset, valid_dataset), 
    batch_size = config.batch_size,
    device = device,
    sort = False, # whether sort the whole dataset with sortkey
    shuffle = True,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True, #sort by length for padding
    repeat = False)

test_iterator = data.Iterator(
    test_dataset,
    batch_size = 64,
    device = device, 
    sort = False, 
    shuffle = False,
#     sort_key = lambda x: len(x.text),
    sort_within_batch = False, # don't wanna sort in testing set
    repeat = False)

In [643]:
train_dataset[0].text[:5]

['a', 'very', 'well', 'made', 'film']

In [644]:
TEXT.vocab.freqs.most_common(1)

[('the', 21075)]

In [645]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f1ba8a3c048>, {'neg': 0, 'pos': 1})


In [646]:
# m = nn.AdaptiveAvgPool2d((1,None))
# input = torch.randn(123, 2, 256).permute(1,0,2)
# output = m(input).squeeze(1)

In [647]:
# output[0][1]

# Define model

In [688]:
model = baseline_lstm_model.BaselineLstm(vocab_size=len(TEXT.vocab), 
                                            embed_size=300, 
                                            hidden_size=32, 
                                            output_dim=1,
                                            nlayers=1,
                                            bidirectional=True,
                                            lstm_dropout=0,
                                            dropout=0.6,
                                            pad_idx=TEXT.vocab.stoi[TEXT.pad_token],
                                            train_embedding=False)

In [677]:
# model = oh_lstm_model.OhLstm(vocab_size=len(TEXT.vocab), 
#                                             embed_size=300, 
#                                             hidden_size=32, 
#                                             output_dim=1,
#                                             nlayers=1,
#                                             bidirectional=True,
#                                             lstm_dropout=0,
#                                             dropout=0.6,
#                                             pad_idx=TEXT.vocab.stoi[TEXT.pad_token],
#                                             train_embedding=False)

In [678]:
# model = fasttext_model.Fasttext(vocab_size=len(TEXT.vocab), 
#                                             embed_size=300,
#                                             output_dim=1,
#                                             dropout=0,
#                                             pad_idx=TEXT.vocab.stoi[TEXT.pad_token],
#                                             train_embedding=True)

In [689]:
model

BaselineLstm(
  (embedding): Embedding(22557, 300, padding_idx=1)
  (lstm): LSTM(300, 32, bidirectional=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.6)
)

In [690]:
pretrained_embeddings = TEXT.vocab.vectors

# load embedding
model.embedding.weight.data.copy_(pretrained_embeddings)
# # init unk token
# UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
# model.embedding.weight.data[UNK_IDX] = torch.zeros(200)
# model.embedding.weight.data[TEXT.vocab.stoi[TEXT.pad_token]] = torch.zeros(200)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [ 0.2720, -0.0620, -0.1884,  ...,  0.1302, -0.1832,  0.1323],
        ...,
        [ 0.9868, -0.0743,  0.3943,  ..., -0.9680, -0.8778,  0.9756],
        [ 0.3286, -0.0345,  0.0263,  ..., -1.5719,  0.0982,  0.7309],
        [ 0.1896,  0.1672,  0.3407,  ..., -0.3786, -0.0278, -0.1949]])

In [691]:
optimizer = optim.Adam(model.parameters(), lr=1e-03)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, weight_decay=5e-4)
criterion = nn.BCEWithLogitsLoss()

best_epoch, best_vali_loss, starting_epoch = 0, 400, 0

In [692]:
baseline_lstm_model.run(model, 
                        optimizer, 
                        criterion, 
                        train_iterator, 
                        valid_iterator, 
                        best_epoch=best_epoch, 
                        best_vali_loss=best_vali_loss, 
                        DEVICE=device, 
                        start_epoch=starting_epoch)

### Epoch     0 
Epoch: 0	Batch: 200	Avg-Loss: 0.6877	Avg-Acc: 0.5250 
Epoch: 0	Batch: 400	Avg-Loss: 0.6886	Avg-Acc: 0.5425 
Epoch: 0	Batch: 600	Avg-Loss: 0.6667	Avg-Acc: 0.5825 
Epoch: 0	Batch: 800	Avg-Loss: 0.6430	Avg-Acc: 0.6275 
Train Loss: 0.7617	Train Acc: 0.5581	Val Loss: 0.7450	Val Acc: 0.5875 
Epoch time used:  44.034775495529175 s 
### Epoch     1 
Epoch: 1	Batch: 200	Avg-Loss: 0.6075	Avg-Acc: 0.6975 
Epoch: 1	Batch: 400	Avg-Loss: 0.6278	Avg-Acc: 0.6500 
Epoch: 1	Batch: 600	Avg-Loss: 0.6211	Avg-Acc: 0.6400 
Epoch: 1	Batch: 800	Avg-Loss: 0.5851	Avg-Acc: 0.7150 
Train Loss: 0.5511	Train Acc: 0.7350	Val Loss: 0.5696	Val Acc: 0.7200 
Epoch time used:  36.316930532455444 s 
### Epoch     2 
Epoch: 2	Batch: 200	Avg-Loss: 0.5513	Avg-Acc: 0.7200 
Epoch: 2	Batch: 400	Avg-Loss: 0.5223	Avg-Acc: 0.7400 
Epoch: 2	Batch: 600	Avg-Loss: 0.5500	Avg-Acc: 0.7125 
Epoch: 2	Batch: 800	Avg-Loss: 0.4973	Avg-Acc: 0.7675 
Train Loss: 0.5591	Train Acc: 0.7156	Val Loss: 0.7530	Val Acc: 0.6475 
Epoch ti

KeyboardInterrupt: 

# Prediction

In [673]:
for epoch in [4]:
    # checkpoint = torch.load("checkpoint.pt")
    model_prediction = oh_lstm_model.OhLstm(vocab_size=len(TEXT.vocab), 
                                            embed_size=300, 
                                            hidden_size=32, 
                                            output_dim=1,
                                            nlayers=1,
                                            bidirectional=True,
                                            lstm_dropout=0,
                                            dropout=0.6,
                                            pad_idx=TEXT.vocab.stoi[TEXT.pad_token],
                                            train_embedding=False)
    # proceeding from old models
    model_path = os.path.join(paths.output_path, config.model_prefix+str(epoch)+'.pth.tar')
    print("=> loading checkpoint '{}'".format(model_path))
    checkpoint = torch.load(model_path)
    starting_epoch = checkpoint['epoch']
    # best_vali_acc = checkpoint['best_vali_acc']
    val_loss: checkpoint['val_loss']
    val_acc: checkpoint['val_acc']
    model_state_dict = checkpoint['model_state_dict']
    model_prediction.load_state_dict(model_state_dict)
    # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    best_vali_loss = checkpoint['best_vali_loss']
    best_epoch = checkpoint['best_epoch']
    print("=> loaded checkpoint '{}' (epoch {})"
          .format(model_path, checkpoint['epoch']))
    # del checkpoint, model_state_dict

    best_epoch

    model_prediction.cuda()

    prediction = baseline_lstm_model.predict(model_prediction, test_iterator, device)

=> loading checkpoint '../outputs/baseline_lstm_4.pth.tar'
=> loaded checkpoint '../outputs/baseline_lstm_4.pth.tar' (epoch 4)
0


In [674]:
len(prediction)

2000

In [675]:
prediction_itos = [LABEL.vocab.itos[int(idx)] for idx in prediction]

In [676]:
with open(os.path.join(paths.output_path, 'heldout_pred_nn_val89.txt'), 'w', encoding='utf-8') as f:
    [f.write(prediction_string+'\n') for prediction_string in prediction_itos]

# Temp

In [139]:
# collate fn lets you control the return value of each batch
# for packed_seqs, you want to return your data sorted by length
def collate_lines_for_test(seq_list, lens):
    inputs = seq_list.permute(1,0).cpu().numpy()
#     lens = [len(seq) for seq in inputs]
    # sort by length
    seq_order = sorted(range(len(lens)), key=lens.__getitem__, reverse=True)
    ordered_inputs = torch.tensor([inputs[i] for i in seq_order]).permute(1,0).cuda()
    reverse_order = sorted(range(len(lens)), key=seq_order.__getitem__, reverse=False)
    return ordered_inputs,reverse_order

In [122]:
original_inputs, original_lengths = next(iter(test_iterator)).text

In [127]:
original_inputs.permute(1,0)

array([[   11,    34,     6, ...,     1,     1,     1],
       [ 4533,  3995,    91, ..., 13735,   185,     4],
       [   51,   325,     6, ...,     1,     1,     1],
       ...,
       [   24,    20,   627, ...,     1,     1,     1],
       [  153,    15,     6, ...,     1,     1,     1],
       [ 3184,    81,    11, ...,     1,     1,     1]])

In [140]:
test_inputs, reverse_order = collate_lines_for_test(original_inputs, original_lengths)

In [148]:
torch.eq(test_inputs.permute(1,0)[reverse_order].permute(1,0),original_inputs).all()

tensor(1, device='cuda:0', dtype=torch.uint8)